<a href="https://colab.research.google.com/github/JoelYanotka/text-summary-to-speech/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/huggingface/transformers sentencepiece datasets

In [ ]:
!pip install wikipedia

In [3]:
from transformers import pipeline

In [ ]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

In [57]:
import re

import wikipedia
from urllib.request import urlopen
from bs4 import BeautifulSoup

def summarize_page(wiki_title):
    # Get a Wikipedia page by its title
    wiki = wikipedia.page(wiki_title)

    # Get a Wikipedia page by its url and make a soup
    source = urlopen(wiki.url).read()
    soup = BeautifulSoup(source,'lxml')

    # page will contain the list of sections in the page, as delimited by html headlines
    # Add a list with the page title as the first element and the summary as the second
    page = [[soup.find('h1').get_text(), wiki.summary]]

    for header in soup.find_all(['h2', 'h3']):
        header_name = header.get_text().replace('[edit]', '')
        if header.get_text() == 'Contents':
            continue
        if 'References' in header.get_text():
            break
        # Every element in the section list will be a list with the name of the
        # headline as the first element and the paragraphs as the next elements
        section = [header_name]
        for elem in header.next_siblings:
            # Stop at next header
            if elem.name and elem.name.startswith('h'):
                break
            if elem.name == 'p':
                # re.sub eliminate references
                paragraph = re.sub(r'\[.*?\]+', '', elem.get_text())\
                    .replace('\n', '')\
                    .replace(u'\xa0', ' ')
                section.append(paragraph)
        page.append(section)

    summaries = []
    for i, section in enumerate(page):
        if len(section) > 1:
            for j, paragraph in enumerate(section[1:]):
                lenght = len(paragraph.split())
                summary = summarizer(paragraph, max_length=lenght, min_length=2, do_sample=False)
                summaries.append(summary[0]['summary_text'])
                print(f'\rSection: {i+1}/{len(page)}\tParagraph: {j+1} / {len(section)}', end='')
    print('')
    return summaries

In [58]:
wiki_title = 'Sceloporus_virgatus'

summarized_text = summarize_page(wiki_title)

Section: 12/12	Paragraph: 1 / 2


In [64]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan
from datasets import load_dataset
import torch
import soundfile as sf


processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

def text2audio(text, output_name='output.wav', processor=processor, model=model, vocoder=vocoder):

    inputs = processor(text=text, return_tensors="pt")

    # load xvector containing speaker's voice characteristics from a dataset
    embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
    speaker_embeddings = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

    speech = model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)

    sf.write(output_name, speech.numpy(), samplerate=16000)

In [ ]:
for i, line in enumerate(summarized_text):
    text2audio(text=line, output_name=f'output_{i:02d}.wav')

In [70]:
import os
import wave

infiles = [f for f in os.listdir() if f.startswith('output_')]
infiles.sort()
outfile = "output.wav"

data= []
for infile in infiles:
    w = wave.open(infile, 'rb')
    data.append([w.getparams(), w.readframes(w.getnframes())])
    w.close()
    
output = wave.open(outfile, 'wb')
output.setparams(data[0][0])
for i in range(len(data)):
    output.writeframes(data[i][1])
output.close()